## Практическое задание к уроку 9 по теме "Языковое моделирование".

*Разобраться с моделью генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов.*

В данном задании напишу модель генерации текста аналогичную той, что была  
на уроке. Но буду использовать pytorch, а также устраню ошибку, по которой  
не использовался hidden_state модели при генерации текста, в результате чего  
после предсказания первого символа модель получала на вход только один символ,  
без контекста.

Загрузим библиотеки и датасет:

In [1]:
import numpy as np
import torch
from torch import nn
from torchinfo import summary

In [2]:
RANDOM_STATE = 29

In [3]:
with open('../../Теория/Lesson_9/evgenyi_onegin.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
text[:500]

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                                Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Святой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высо'

In [5]:
len(text)

286984

Создадим словарь символов:

In [6]:
vocab = sorted(set(text))

In [7]:
VOCAB_SIZE = len(vocab)
VOCAB_SIZE

131

Создадим словари маппинга символов и их индексов:

In [8]:
char2idx = {c: i for i, c in enumerate(vocab)}
idx2char = np.array(vocab)

text_idx = np.array([char2idx[c] for c in text])

In [9]:
text_idx[:100]

array([ 71, 110, 104, 109, 116,  99, 112, 103, 115,   1,  87, 104, 115,
       102, 104, 104, 101, 107, 122,   1,  85, 118, 123, 109, 107, 112,
         0,   0,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,  76, 101, 102, 104, 112,
       107, 108,   1,  84, 112, 104, 102, 107, 112,   0,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1])

In [10]:
text_idx.shape

(286984,)

Напишем класс датасета, в котором будем разбивать текст  
на куски заданной последовательности, а также сделаем  
смещение входной последовательности для получения таргета.  
То есть предсказывать будем следующий символ.

In [11]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, text_idx, seq_length=100):
        
        remainder = len(text_idx) % seq_length
        self.text = text_idx[:-remainder]
        self.text = self.text.reshape(-1, seq_length)
        self.text = torch.from_numpy(self.text)
    
    def __getitem__(self, index):
        return self.text[index][:-1], self.text[index][1:]
    
    def __len__(self):
        return len(self.text)

Создадим даталоадер:

In [12]:
BATCH_SIZE = 64
SEQ_LENGTH = 100

In [13]:
torch.random.manual_seed(RANDOM_STATE)

dataset = TextDataset(text_idx, SEQ_LENGTH)

loader = torch.utils.data.DataLoader(dataset,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True)

Напишем сеть:

In [14]:
class Net(nn.Module):
    def __init__(self, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embedding = nn.Embedding(VOCAB_SIZE, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, VOCAB_SIZE)
        
    def forward(self, x, h_0=None):
        x = self.embedding(x)
        
        # Вектор скрытого состояния будем использовать при генерации
        # текста, сохранять его, чтобы при предсказании символов не  
        # нужно было каждый раз прогонять всю последовательность
        x, h_n = self.gru(x, h_0) 
        
        # Полносвязный слой получает последовательность и для каждого  
        # её элемента выдаёт последовательность логитов
        x = self.fc(x)
        
        return x.permute(0, 2, 1), h_n

In [15]:
summary(Net(embed_dim=256, hidden_dim=512))

Layer (type:depth-idx)                   Param #
Net                                      --
├─Embedding: 1-1                         33,536
├─GRU: 1-2                               2,758,656
├─Linear: 1-3                            67,203
Total params: 2,859,395
Trainable params: 2,859,395
Non-trainable params: 0

На гиперпараметрах, на которых будем обучать сеть, имеем чуть  
меньше 3 млн. параметров.

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Напишем функцию для обучения сети. Здесь всё стандартно,  
кроме того, что у нас нет валидации:

In [17]:
def train_nn(epochs=5, embed_dim=128, hidden_dim=256, lr=1e-3, return_net=False):
    
    torch.random.manual_seed(RANDOM_STATE)
    torch.backends.cudnn.deterministic = True

    net = Net(embed_dim=embed_dim, hidden_dim=hidden_dim).to(device)

    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        losses = np.array([])

        for inputs, labels in loader:
            net.train()
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)[0]
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            losses = np.append(losses, loss.item())

        print(f'Epoch [{epoch + 1}/{epochs}]. ' \
              f'Loss: {losses.mean():.3f}. ')

    print('Training is finished!')
    if return_net:
        return net

Обучим сеть:

In [18]:
net = train_nn(epochs=50, embed_dim=256, hidden_dim=512, return_net=True)

Epoch [1/50]. Loss: 1.915. 
Epoch [2/50]. Loss: 1.358. 
Epoch [3/50]. Loss: 1.229. 
Epoch [4/50]. Loss: 1.156. 
Epoch [5/50]. Loss: 1.104. 
Epoch [6/50]. Loss: 1.056. 
Epoch [7/50]. Loss: 1.009. 
Epoch [8/50]. Loss: 0.982. 
Epoch [9/50]. Loss: 0.933. 
Epoch [10/50]. Loss: 0.916. 
Epoch [11/50]. Loss: 0.881. 
Epoch [12/50]. Loss: 0.848. 
Epoch [13/50]. Loss: 0.813. 
Epoch [14/50]. Loss: 0.815. 
Epoch [15/50]. Loss: 0.773. 
Epoch [16/50]. Loss: 0.736. 
Epoch [17/50]. Loss: 0.697. 
Epoch [18/50]. Loss: 0.660. 
Epoch [19/50]. Loss: 0.627. 
Epoch [20/50]. Loss: 0.601. 
Epoch [21/50]. Loss: 0.565. 
Epoch [22/50]. Loss: 0.533. 
Epoch [23/50]. Loss: 0.510. 
Epoch [24/50]. Loss: 0.473. 
Epoch [25/50]. Loss: 0.449. 
Epoch [26/50]. Loss: 0.412. 
Epoch [27/50]. Loss: 0.388. 
Epoch [28/50]. Loss: 0.381. 
Epoch [29/50]. Loss: 0.349. 
Epoch [30/50]. Loss: 0.322. 
Epoch [31/50]. Loss: 0.302. 
Epoch [32/50]. Loss: 0.286. 
Epoch [33/50]. Loss: 0.275. 
Epoch [34/50]. Loss: 0.262. 
Epoch [35/50]. Loss: 0.

Напишем функцию для генерации текста:

In [19]:
def generate_text(model, start_string, generated_length=100, temperature=1., randomized=False):
    
    '''
    start_string - начальные символы строки, которую нужно сгенерировать
    
    generated_length - количество символов, которые будем генерировать
    
    temperature - температура, некоторая степень хаоса, влияющая на то,
    как сильно будут влиять распределения логитов на выходе сети на выбор
    соответствующих символов при предсказании. Чем больше значение, тем  
    меньше влияние логитов и тем случайнее предсказание
    
    randomized - генерировать ли каждый раз новую последовательность
    '''
    
    if not randomized:
        torch.random.manual_seed(RANDOM_STATE)
    
    # Переводим последовательность в индексы
    idx_string = torch.IntTensor([char2idx[c] for c in start_string]).to(device)
    
    # Вектор скрытого состояния для слоя RNN. Инициализируем "наном", что  
    # равносильно нулевому вектору
    hidden_state = None 
    
    for _ in range(generated_length):
        
        # На первой итерации пропускаем всю начальную последовательность через модель, 
        # на входе имеем нулевой вектор скрытого состояния. На последующих итерациях пропускаем
        # только последний предсказанный символ, а также накопленную информацию о контексте  
        # в векторе скрытого состояния
        predicted_idx, hidden_state = model(idx_string[None, :], hidden_state)
        
        # Берём последний символ в единственном батче. Получаем его логиты,  
        # где каждый символ словаря имеет определённый скор (score)
        predicted_idx = predicted_idx[0, :, -1]
        
        # Вводим в действие температуру
        predicted_idx /= temperature
        
        # Берём кандидата из распределения логитов
        predicted_idx = torch.distributions.categorical.logits_to_probs(predicted_idx) 
        predicted_idx = torch.multinomial(predicted_idx, num_samples=1)
        
        # В следующей итерации на вход сети подаём предсказанный символ
        idx_string = predicted_idx
        
        # Расшифровываем кандидата через словарь маппинга
        predicted_char = idx2char[predicted_idx]
        
        # Добавляем предсказание к строке
        start_string += predicted_char
        
    return start_string

Посмотрим на предсказание модели. На вход подадим строку из  
стихотворения М.Ю. Лермонтова:

In [20]:
text = 'Белеет парус одинокий '
print(generate_text(net, text, generated_length=1000, temperature=0.5, randomized=False))

Белеет парус одинокий голос летит,
                               И славы селенить бы всеми
                         Замену туский один печальных:
                        За ним сердцем и умом крест.

                                                          XLV

                        Того, почта задержало спясь!
                        Когда б мне быть отцом, супругом
                        Обманчир, когда ночная тень,
                        Приемы скоро приняла!
                        Услышу ль вновь поражен.

                                          XLIII

                          Онегин вышел иногда:
                         Всем сердцем юноша лежит,
                         И все дела, все речи мерит
                         И намара не читал он.

                                   XXXIV

                         Пред ним на стол разговор странность
                        И нам досталось она
                        Он должен быть, без гремене;
                        В глу

В целом с задачей справились, но хочется ещё "покрутить" параметр температуры.  
Что если зададим достаточно большое значение?

In [21]:
print(generate_text(net, text, generated_length=1000, temperature=3., randomized=False))

Белеет парус одинокий фои),
               ЛебаньегНа ДлягАзиБйи Рофйой,
      К       Неgел сГhам Pan.otеbнью,
                Влешеи тесныймА, (Брет дрcвнукЛ
                    ftpilnТ Cs иL РiotчуА ЦилыщихХты-Ппит"
                 Я утременУА l25вы
                   ЧЗароИ зwвю берегgа
 Д            Mдnеe!ф -
             К грАди, Лще та прiмал, живъей;
     "    Знячным похв'-vdGoбрeш;
    Бu               Небо был, сквозь вернойut
              Дырено тгнимуX
 ,            тяпижуО чтюФт.
             Х zrа,щийЖки Цицеримых
              N плючки дизто0оeДрагии
            А плАмШpУ Татицы t7ва4,
                - Бариты СЮиn} живык.
              ХовЬаяческийФяГ
И           Н е тежкаЦи,БЖь Форны,
                   Улча, пламеRнщю цЖвальных щh2еt
         Бы Смлогих дводил.
     .         Вщает бж! бuсВрадцейный Юльзыq}
              И зналдях флата; Журант.
  "               Пьюм ийБегПе ейжила:
                 СъягСенул - чест вет?Ча-Хо;
             с асce, ШриВка-Бурш!), -

Получили хаос, как и должно было быть, ведь теперь величина логитов всех  
символов становится очень близкой. Теперь попробуем малое значение:

In [22]:
print(generate_text(net, text, generated_length=1000, temperature=0.0001, randomized=False))

Белеет парус одинокий голос лир,
                         Не привлекла б она очей.
                         Да стол подвинь; я скоро лягкой,
                         Предчувствия теснили грудь.
                        Простите, игры золотые!
                        Он так привык перевестила;
                        Стихи введут в употреблен;
                        Конечно, не один Евгений
                        На прелестных помительной -
                        Волшебный дев умел судит
                         Себе присвоить ум чужой;
                        От жад, покорны старинной
                        Они не страдали нравими
                            Меняю милых полковая!
                         Да стол подвинь; я скоро лягу;
                        И там уж ты не был того: для всех,
                                                                                                                                                                                                 

Интересный факт здесь в том, что теперь влияние сэмплирования по логитам сведено  
к минимуму. Почти всегда будут выбираться кандидаты с самой высокой вероятностью,  
и гиперпараметр randomized теряет своё действие:

In [23]:
generate_text(net, text, generated_length=1000, temperature=0.0001, randomized=False) == \
generate_text(net, text, generated_length=1000, temperature=0.0001, randomized=True)

True

Для сравнения, при "нормальном" значении температуры:

In [24]:
generate_text(net, text, generated_length=1000, temperature=0.5, randomized=False) == \
generate_text(net, text, generated_length=1000, temperature=0.5, randomized=True)

False